In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle as pk
import nltk
import seaborn as sns
import matplotlib.pyplot as plt



export_summaries_path = Path('output/summaries/methods_per_text')
export_summaries_path.mkdir(parents=True, exist_ok=True)

In [ ]:

dfs = []
for file in export_summaries_path.glob('*.csv'):
    df = pd.read_csv(file)
    generation_method, dataset = file.stem.split('-_-')[:2]
    
    df['metadata/Generation'] = generation_method
    df['metadata/Dataset'] = dataset
    
    dfs.append(df)
    
df = pd.concat(dfs)

df = df.drop([c for c in df.columns if "Unnamed:" in c], axis=1)

del dfs

def replace_abstractive(x):
    if "abstractive" in x:
        return "extractive_sentences"
    else:
        return x

df['metadata/Generation'] = df['metadata/Generation'].apply(replace_abstractive)



In [ ]:
df = df[~(df['Method'].str.contains('Lead')).fillna(False)]



In [ ]:
ddf = df.copy()
ddf['proba_of_success'] = ddf['proba_of_success'].apply(np.exp)

discriminativity = ddf.groupby(['metadata/Generation', 'metadata/reranking_model', 'Method'])[['proba_of_success', 'LM Perplexity']].agg(['mean']).droplevel(1, axis=1).sort_values('proba_of_success', ascending=False).reset_index()
discriminativity

In [ ]:

ddf = ddf.sort_values('proba_of_success', ascending=False)
sns.catplot(data=ddf, y='proba_of_success', x='Method', hue="metadata/Generation", kind='bar', col='metadata/reranking_model')

In [ ]:

fig, ax = plt.subplots(1, 1)

paretto = get_pareto_points(discriminativity[['proba_of_success', 'LM Perplexity']].values)

ax.plot(paretto[:, 1], paretto[:, 0], c='purple', linewidth=5, linestyle="--", label="Pareto front")
sns.scatterplot(data=discriminativity, y='proba_of_success', x='LM Perplexity', hue="Method", s=200, alpha=0.8, style='metadata/reranking_model')
plt.xlim(-70, 0)

        

def get_pareto_points(data):
    # data : [N, 2]
    
    optima = []
    for p in data:
        x, y = p
        if len([p2 for p2 in data if p2[0] > p[0] and p2[1] > p[1]]) == 0:
            optima.append(p)
            
    return np.array(optima)



            

    
    
    

In [ ]:

ddf = df.copy().drop('level_0', axis=1)
ddf['proba_of_success'] = -ddf['proba_of_success']
ddf['LM Perplexity'] = -ddf['LM Perplexity']
ddf = ddf.reset_index()




sns.displot(data=ddf, x='LM Perplexity', y='proba_of_success', hue="Method", kind='kde')
# plt.xscale('log')
#plt.yscale('log')

plt.xlim(0, 100)
plt.ylim(-2, 3)



In [ ]:
sns.scatterplot(data=df, x='LM Perplexity', y='proba_of_success', hue="Method")
